### Section bodies combined by score order top 1000 words

#### Steps:

1. Get the relevant sections and their position in the report
2. Order the sections by relevance score
3. Extract the body of section and keep on concatenating
4. Extract top 1000 words
5. If no relevant sections -> Write the complete file as summary with top 1000 words
6. Compare the system summary with each gold summary and get the rouge score. Take average of all gold summaries for the respective file.
7. Take average of all the scores for all the files.

In [9]:
import pandas as pd
import pickle
import os

from extract_section_body import extract_section_body
from rouge_evaluation import get_rouge_scores

In [10]:
VALIDATION_DATASET = True
TEST_DATASET = False

In [11]:
if VALIDATION_DATASET:
    dir_ = 'C:/My folders/Deva paul/AI/SEM 5/356 NLP/NLP Project/New folder/FNS2023_Datasets/English/validation/'
    toc_loc_pkl_file_path = 'C:/My folders/Deva paul/AI/SEM 5/356 NLP/NLP Project/New folder/FNS2023_Datasets/English/validation/out/valid_toc_loc.pkl'
    df_predicted_path = 'C:/My folders/Deva paul/AI/SEM 5/356 NLP/NLP Project/New folder/2_Section_Classification/out/validation_df_predicted.pkl'

if TEST_DATASET:
    dir_ = '../../../Dataset/FNS2022/English/testing/'
    toc_loc_pkl_file_path = '../../../Dataset/Annotated_Dataset/test_toc_loc.pkl'
    df_predicted_path = '../../FNP2022/2_Section_Classification/out/test_df_predicted.pkl'

annual_reports_dir = "annual_reports"
gold_summary_dir = "gold_summaries"
system_summary_dir = 'section_combined_by_score_top_1000'
team_name = 'IT-356'
dir_

'C:/My folders/Deva paul/AI/SEM 5/356 NLP/NLP Project/New folder/FNS2023_Datasets/English/validation/'

In [12]:
df_predicted = pickle.load(open(df_predicted_path, 'rb'))
df_predicted

,file_id,toc_section,toc_section_pos,toc_section_len,is_section_in_summary,toc_section_cleaned,pred,False,True
0,30777,Financial and operational highlights,161,22,0,financi oper highlight,1,0.407563,0.592437
1,30777,Strategic report,183,6,1,strateg report,0,0.932972,0.067028
2,30777,Global network,189,11,0,global network,0,0.756613,0.243387
3,30777,Chairman’s statement,200,4,1,chairman statement,1,0.018573,0.981427
4,30777,Chief Executive’s review,204,4,1,chief execut review,1,0.007140,0.992860
...,...,...,...,...,...,...,...,...,...
10547,4162,"to 110,",21914,1104,0,,0,0.983343,0.016657
10548,4162,and 117,23018,1104,0,,0,0.985658,0.014342
10549,4162,to 116,24122,1104,0,,0,0.987655,0.012345
10550,4162,to 122,25226,5440,0,,0,0.987178,0.012822


In [13]:
def get_relevant_sections_with_score(file_id):
    df_dict = df_predicted[(df_predicted.file_id == int(file_id)) & (df_predicted.pred == 1)][['toc_section', 'True']].to_dict('list')
    section_score_dict = {}
    toc_sections = df_dict['toc_section']
    section_scores = df_dict['True']
    for i in range(len(toc_sections)):
        section_score_dict[toc_sections[i]] = section_scores[i]
    
    result_dict = {k: v for k, v in sorted(section_score_dict.items(), key=lambda item: item[1], reverse=True)}
    releavnt_sections = list(result_dict.keys())
    return releavnt_sections, result_dict

In [14]:
num_file = 0
os.makedirs(system_summary_dir)
for file in os.listdir(os.path.join(dir_, annual_reports_dir)):
    try:
        #print("Processing File Number: ", num_file)
        num_file = num_file +1 
        file_id = file.split('.')[0]
        relevant_sections_with_score = get_relevant_sections_with_score(file_id)
        # Section order is maintained
        relevant_sections = relevant_sections_with_score[0]
        #print(file_id, relevant_sections)
        summary = ""
        if relevant_sections:
            #print('Relevant Section Found in ', file_id)
            for section in relevant_sections:
                summary = summary + " " +extract_section_body(file_id, section, dir_, annual_reports_dir, toc_loc_pkl_file_path)
            summary_split = summary.split(' ')
            number_of_words = len(summary_split)
            #print(number_of_words)
            if number_of_words > 1000:
                summary = " ".join(summary_split[:1000])
    
        else:
            #print('Relevant Section Not Found in ', file_id)
            summary = open(os.path.join(dir_, annual_reports_dir, file), "r", encoding="utf-8").read()
            summary_split = summary.split(' ')
            number_of_words = len(summary_split)
            #print(number_of_words)
            if number_of_words > 1000:
                summary = " ".join(summary_split[:1000])
        
        with open(os.path.join(system_summary_dir, file_id+'_'+team_name+'.txt'), 'w', encoding='utf-8') as f:
            f.write(str(summary))
            
        if ".DS_Store" in file:
            continue
    except Exception as e:
        print(file, e)

In [15]:
if VALIDATION_DATASET:
    gold_summary_dir_ =  os.path.join(dir_, gold_summary_dir)
    rouge_scores = get_rouge_scores(system_summary_dir, gold_summary_dir_)
    rouge_scores

Processing File Number:  0
Processing File Number:  50
Processing File Number:  100
Processing File Number:  150
Processing File Number:  200
Processing File Number:  250
Processing File Number:  300
Processing File Number:  350
Processing File Number:  400
Number of files processed:  413


In [16]:
rouge_scores

{'rouge-1': {'p': 0.5357845387362559,
  'r': 0.5079333324073311,
  'f': 0.44754709786025393},
 'rouge-2': {'p': 0.34370825188104115,
  'r': 0.32471788901582177,
  'f': 0.2855073745507965}}